# 3. Machine Learning for Classification

### Dataset

In this homework, we will use the Car price dataset. Download it from [here](https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv).
https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv

## 3.2 Data preparation

* Download the data, read it with pandas
* Look at the data
* Make column names and values look uniform
* Check if all the columns read correctly
* Check if the churn variable needs any preparation

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [2]:
data = 'https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv'

In [3]:
!wget $data -O car_price.csv 

--2023-09-30 18:14:34--  https://raw.githubusercontent.com/alexeygrigorev/mlbookcamp-code/master/chapter-02-car-price/data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1475504 (1.4M) [text/plain]
Saving to: ‘car_price.csv’

car_price.csv       100%[===================>]   1.41M  2.32MB/s    in 0.6s    

2023-09-30 18:14:36 (2.32 MB/s) - ‘car_price.csv’ saved [1475504/1475504]



In [58]:
fc = [
    'Make',
    'Model',
    'Year',
    'Engine HP',
    'Engine Cylinders',
    'Transmission Type',
    'Vehicle Style',
    'highway MPG',
    'city mpg'
]
all_c = fc + ['MSRP']
df = pd.read_csv('car_price.csv', usecols= all_c)
df.head()

,Make,Model,Year,Engine HP,Engine Cylinders,Transmission Type,Vehicle Style,highway MPG,city mpg,MSRP
0,BMW,1 Series M,2011,335.0,6.0,MANUAL,Coupe,26,19,46135
1,BMW,1 Series,2011,300.0,6.0,MANUAL,Convertible,28,19,40650
2,BMW,1 Series,2011,300.0,6.0,MANUAL,Coupe,28,20,36350
3,BMW,1 Series,2011,230.0,6.0,MANUAL,Coupe,28,18,29450
4,BMW,1 Series,2011,230.0,6.0,MANUAL,Convertible,28,18,34500


We'll keep working with the MSRP variable, and we'll transform it to a classification task.
Features

For the rest of the homework, you'll need to use only these columns:

    Make,
    Model,
    Year,
    Engine HP,
    Engine Cylinders,
    Transmission Type,
    Vehicle Style,
    highway MPG,
    city mpg
    
Data preparation
Select only the features from above and transform their names using next line:

data.columns = data.columns.str.replace(' ', '_').str.lower()

Fill in the missing values of the selected features with 0.
Rename MSRP variable to price.


In [59]:
df = df.rename(columns={'MSRP':'price'})

In [60]:
df.columns = df.columns.str.lower().str.replace(' ', '_')

categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ', '_')



In [61]:
df.fillna(0, inplace = True)

In [62]:
df.head().T

,0,1,2,3,4
make,bmw,bmw,bmw,bmw,bmw
model,1_series_m,1_series,1_series,1_series,1_series
year,2011,2011,2011,2011,2011
engine_hp,335.0,300.0,300.0,230.0,230.0
engine_cylinders,6.0,6.0,6.0,6.0,6.0
transmission_type,manual,manual,manual,manual,manual
vehicle_style,coupe,convertible,coupe,coupe,convertible
highway_mpg,26,28,28,28,28
city_mpg,19,19,20,18,18
price,46135,40650,36350,29450,34500



### Question 1

What is the most frequent observation (mode) for the column `transmission_type`?

- `AUTOMATIC` - this one
- `MANUAL`
- `AUTOMATED_MANUAL`
- `DIRECT_DRIVE`

In [63]:
df.groupby('transmission_type').size().sort_values(ascending=False)

transmission_type
automatic           8266
manual              2935
automated_manual     626
direct_drive          68
unknown               19
dtype: int64

### Question 2

Create the [correlation matrix](https://www.google.com/search?q=correlation+matrix) for the numerical features of your dataset. 
In a correlation matrix, you compute the correlation coefficient between every pair of features in the dataset.

What are the two features that have the biggest correlation in this dataset?

- `engine_hp` and `year`
- `engine_hp` and `engine_cylinders`
- `highway_mpg` and `engine_cylinders`
- `highway_mpg` and `city_mpg

Answer: `highway_mpg` and `city_mpg

In [113]:
numerical = ['engine_hp', 'engine_cylinders', 'highway_mpg', 'city_mpg', 'year']
df_corr=df[numerical]
print(df_corr.corr())

                  engine_hp  engine_cylinders  highway_mpg  city_mpg      year
engine_hp          1.000000          0.774851    -0.415707 -0.424918  0.338714
engine_cylinders   0.774851          1.000000    -0.614541 -0.587306 -0.040708
highway_mpg       -0.415707         -0.614541     1.000000  0.886829  0.258240
city_mpg          -0.424918         -0.587306     0.886829  1.000000  0.198171
year               0.338714         -0.040708     0.258240  0.198171  1.000000


### Make `price` binary

* Now we need to turn the `price` variable from numeric into a binary format.
* Let's create a variable `above_average` which is `1` if the `price` is above its mean value and `0` otherwise.

### Split the data

* Split your data in train/val/test sets with 60%/20%/20% distribution.
* Use Scikit-Learn for that (the `train_test_split` function) and set the seed to `42`.
* Make sure that the target value (`above_average`) is not in your dataframe.

    

In [74]:
avg_price = df.price.mean()
avg_price
df['above_average'] = np.where(df.price>avg_price, 1, 0)
df.head()

,make,model,year,engine_hp,engine_cylinders,transmission_type,vehicle_style,highway_mpg,city_mpg,price,above_average
0,bmw,1_series_m,2011,335.0,6.0,manual,coupe,26,19,46135,1
1,bmw,1_series,2011,300.0,6.0,manual,convertible,28,19,40650,1
2,bmw,1_series,2011,300.0,6.0,manual,coupe,28,20,36350,0
3,bmw,1_series,2011,230.0,6.0,manual,coupe,28,18,29450,0
4,bmw,1_series,2011,230.0,6.0,manual,convertible,28,18,34500,0


In [68]:
from sklearn.model_selection import train_test_split

In [75]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [76]:
len(df_train), len(df_val), len(df_test)

(7148, 2383, 2383)

In [77]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)


### Question 3

* Calculate the mutual information score between `above_average` and other categorical variables in our dataset. 
  Use the training set only.
* Round the scores to 2 decimals using `round(score, 2)`.

Mutual information - concept from information theory, it tells us how much 
we can learn about one variable if we know the value of another

* https://en.wikipedia.org/wiki/Mutual_information

Which of these variables has the lowest mutual information score?
  
- `make`
- `model`
- `transmission_type`
- `vehicle_style`

Answer: transmission_type    

In [82]:
categorical = [
    'make',
    'model',
    'transmission_type',
    'vehicle_style'
]

In [83]:
df_full_train[categorical].nunique()

make                  48
model                902
transmission_type      5
vehicle_style         16
dtype: int64

In [88]:
from sklearn.metrics import mutual_info_score



In [89]:
def mutual_info_above_average_score(series):
    return mutual_info_score(series, df_full_train.above_average)

In [90]:
mi = df_full_train[categorical].apply(mutual_info_churn_score)
mi.sort_values(ascending=False)

model                0.460994
make                 0.238724
vehicle_style        0.083390
transmission_type    0.020884
dtype: float64


### Question 4

* Now let's train a logistic regression.
* Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
* Fit the model on the training dataset.
    - To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
    - `model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)`
* Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

What accuracy did you get?

- 0.60
- 0.72
- 0.84
- 0.95

Answer: 0.95


In [91]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

In [131]:
dv = DictVectorizer(sparse=False)

train_dict = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)

val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)

In [132]:

model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

LogisticRegression(C=10, max_iter=1000, random_state=42, solver='liblinear')

In [133]:
model.intercept_[0]

-0.09947066871642177

In [134]:
model.coef_[0].round(3)

array([ 1.900e-02, -7.900e-02,  3.400e-02,  8.000e-02,  1.006e+00,
        6.840e-01,  2.520e-01,  2.342e+00,  8.800e-02,  2.000e+00,
        0.000e+00, -4.380e-01,  1.970e+00, -1.371e+00, -1.270e+00,
       -2.913e+00,  1.500e-01, -2.160e-01, -1.401e+00,  1.920e-01,
       -7.370e-01, -1.210e+00, -9.000e-03, -2.117e+00,  1.460e-01,
       -1.478e+00,  4.000e-03,  1.855e+00,  1.157e+00,  1.080e+00,
        2.432e+00,  4.490e-01,  2.000e-03, -1.146e+00,  0.000e+00,
        1.166e+00, -1.002e+00, -8.880e-01, -6.270e-01, -5.000e-02,
       -1.718e+00,  1.482e+00,  3.760e-01,  9.800e-01, -1.070e-01,
        1.170e-01, -1.653e+00, -1.072e+00,  1.664e+00, -5.960e-01,
       -6.250e-01,  9.480e-01, -8.200e-02, -1.700e-02, -4.000e-03,
       -3.500e-01, -2.000e-03, -4.140e-01, -1.000e-03, -1.000e-03,
       -1.000e-03, -3.000e-01, -1.360e-01, -2.400e-01, -1.270e-01,
       -4.500e-02, -7.400e-02, -5.000e-02, -0.000e+00, -4.000e-03,
       -4.690e-01,  6.600e-02, -5.400e-02,  5.000e-02,  5.230e

In [135]:
y_pred = model.predict_proba(X_val)[:, 1]


In [136]:
decision = (y_pred >= 0.5)

In [137]:
accuracy = (y_val == decision).mean()
accuracy

0.9387326898866974

### Question 5 

* Let's find the least useful feature using the *feature elimination* technique.
* Train a model with all these features (using the same parameters as in Q4).
* Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
* For each feature, calculate the difference between the original accuracy and the accuracy without the feature. 

Which of following feature has the smallest difference?

- `year`
- `engine_hp`
- `transmission_type`
- `city_mpg`

> **Note**: the difference doesn't have to be positive

answer: transmission_type

In [141]:
fs_excl = ['year',    'engine_hp',    'transmission_type',    'city_mpg'] # features to exclude from model

model = LogisticRegression(solver='liblinear', C=10, max_iter=1000, random_state=42)
f_lst = categorical + numerical

for f_excl in fs_excl:
    f_lst_minus= f_lst.copy()
    f_lst_minus.remove(f_excl)
  
    train_dict = df_train[f_lst_minus].to_dict(orient='records')
    X_train = dv.fit_transform(train_dict)

    val_dict = df_val[f_lst_minus].to_dict(orient='records')
    X_val = dv.transform(val_dict)
    model.fit(X_train, y_train)
    y_pred = model.predict_proba(X_val)[:, 1]
    decision = (y_pred >= 0.5)
    acc_minus = (y_val == decision).mean()
    print(f'{f_excl=}: {accuracy=} , acc_minus diff: {(accuracy - acc_minus)*100}')





f_excl='year': accuracy=0.9387326898866974 , acc_minus diff: -0.9651699538396952
f_excl='engine_hp': accuracy=0.9387326898866974 , acc_minus diff: 1.4687368862777994
f_excl='transmission_type': accuracy=0.9387326898866974 , acc_minus diff: -0.5874947545111198
f_excl='city_mpg': accuracy=0.9387326898866974 , acc_minus diff: -0.7553503986571508


### Question 6

* For this question, we'll see how to use a linear regression model from Scikit-Learn.
* We'll need to use the original column `price`. Apply the logarithmic transformation to this column.
* Fit the Ridge regression model on the training data with a solver `'sag'`. Set the seed to `42`.
* This model also has a parameter `alpha`. Let's try the following values: `[0, 0.01, 0.1, 1, 10]`.
* Round your RMSE scores to 3 decimal digits.

Which of these alphas leads to the best RMSE on the validation set?

- 0
- 0.01
- 0.1
- 1
- 10

In [143]:

f_lst = categorical + numericalalpha
alphas = [0, 0.01, 0.1, 1, 10]
for alpha in alphas:
    model = LogisticRegression(solver='sag', C=10, max_iter=1000, random_state=42, alpha=alpha)
    train_dict = df_train[f_lst].to_dict(orient='records')
    X_train = dv.fit_transform(train_dict)

    val_dict = df_val[f_lst].to_dict(orient='records')
    X_val = dv.transform(val_dict)
    model.fit(X_train, y_train)
    y_pred = model.predict_proba(X_val)[:, 1]
    decision = (y_pred >= 0.5)
    accuracy = (y_val == decision).mean()
    print(f'{accuracy=}')


NameError: name 'numericalalpha' is not defined